In [1]:
import ray

# Connect to the Ray cluster
ray.init(address='auto')

@ray.remote
def my_function(x):
    return x * x

# Run the function in parallel
futures = [my_function.remote(i) for i in range(4)]
results = ray.get(futures)
print(results)

2024-11-20 15:20:59,780	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-20 15:21:03,619	INFO worker.py:1634 -- Connecting to existing Ray cluster at address: 129.113.120.144:6379...
2024-11-20 15:21:03,675	INFO worker.py:1810 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


[0, 1, 4, 9]


In [ ]:
#import ray
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Initialize Ray
#ray.init(address='auto')

# Load the Iris dataset
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
data['species'] = iris.target
data['species'] = data['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# Split the data
X = data.drop('species', axis=1)
y = data['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

@ray.remote
def train_model(X_train, y_train):
    from sklearn.tree import DecisionTreeClassifier
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model

# Split the data into chunks for parallel processing
X_train_chunks = np.array_split(X_train, 4)
y_train_chunks = np.array_split(y_train, 4)

# Train models in parallel
futures = [train_model.remote(X_chunk, y_chunk) for X_chunk, y_chunk in zip(X_train_chunks, y_train_chunks)]
models = ray.get(futures)

@ray.remote
def evaluate_model(model, X_test, y_test):
    from sklearn.metrics import accuracy_score
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Evaluate models in parallel
evaluation_futures = [evaluate_model.remote(model, X_test, y_test) for model in models]
accuracies = ray.get(evaluation_futures)
print("Accuracies:", accuracies)


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist


# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Define a function to create and train the model
@ray.remote
def train_model(train_images, train_labels):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5, batch_size=64)
    return model

# Train the model in parallel
future = train_model.remote(train_images, train_labels)
model = ray.get(future)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

(my_function pid=21860) 2024-11-20 15:21:36.599252: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(my_function pid=21860) 2024-11-20 15:21:36.599252: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(my_function pid=21860) 2024-11-20 15:21:36.599252: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(my_function pid=21860) 2024-11-20 15:21:36.59

(train_model pid=21860) Epoch 1/5
(train_model pid=21860) Epoch 1/5
(train_model pid=21860) Epoch 1/5
(train_model pid=21860) Epoch 1/5
(train_model pid=21860) 
(train_model pid=21860)   1/938 ━━━━━━━━━━━━━━━━━━━━ 49:17 3s/step - accuracy: 0.0625 - loss: 2.3066
(train_model pid=21860) 
(train_model pid=21860)   1/938 ━━━━━━━━━━━━━━━━━━━━ 49:17 3s/step - accuracy: 0.0625 - loss: 2.3066
(train_model pid=21860) 
(train_model pid=21860)   1/938 ━━━━━━━━━━━━━━━━━━━━ 49:17 3s/step - accuracy: 0.0625 - loss: 2.3066
(train_model pid=21860)   3/938 ━━━━━━━━━━━━━━━━━━━━ 26s 29ms/step - accuracy: 0.1450 - loss: 2.2845
(train_model pid=21860) 
(train_model pid=21860)   1/938 ━━━━━━━━━━━━━━━━━━━━ 49:17 3s/step - accuracy: 0.0625 - loss: 2.3066
(train_model pid=21860)   3/938 ━━━━━━━━━━━━━━━━━━━━ 26s 29ms/step - accuracy: 0.1450 - loss: 2.2845
(train_model pid=21860) 
(train_model pid=21860)   5/938 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step - accuracy: 0.1703 - loss: 2.2799
(train_model pid=21860) 
(train_

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9859 - loss: 0.0434
Test accuracy: 0.9882000088691711
